# A basic training loop

We will progressively work through several different ways to run a training loop on a basic neural net on the MNIST data set.  We will begin with a version that is primarily in pure Python, and then incrementally add more PyTorch functionality.  Using PyTorch modules and methods will make our code more readable, flexible, and easier to use.  The goal of this notebook is to illustrate the usefulness of PyTorch, and how to use it when training a neural net.

Please note, we will import modules and methods directly before we use them (even though this is not the standard way of writing production code) since we want to make it clear **when** we are using various modules, and what the minimal amount of code needed is at each point.

## MNIST data setup

We will use [pathlib](https://docs.python.org/3/library/pathlib.html), the recommended way for dealing with paths in Python 3 and part of the Python 3 standard library.

In [ ]:
from pathlib import Path

DATA_PATH = Path('data')
PATH = DATA_PATH/'mnist'

PATH.mkdir(parents=True, exist_ok=True)

We download the MNIST dataset, a classic dataset consisting of black-and-white images hand-drawn digits (between 0 and 9).

In [ ]:
import requests

URL='http://deeplearning.net/data/mnist/'
FILENAME='mnist.pkl.gz'

if not (PATH/FILENAME).exists():
    content = requests.get(URL+FILENAME).content
    (PATH/FILENAME).open('wb').write(content)

Although our dataset consists of images, they have been stored using pickle, a python-specific format for compressing data sets.

In [ ]:
import pickle, gzip

with gzip.open(PATH/FILENAME, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

It is always a good idea to take a look at your data to make sure it is what you expect it to be.  Here, our training set consists of 50,000 images.  Each image is 28 x 28, and is being stored as a flattened row of length 784 (=28x28)

In [ ]:
x_train.shape

We can use matplotlib and numpy to display the first image in our training set:

In [ ]:
from matplotlib import pyplot
import numpy as np

pyplot.imshow(np.reshape(x_train[0], (28,28)), cmap="gray")

Now we will import PyTorch.  We will be storing our data as *tensors*.  You can think of a *tensor* as a multidimensional matrix.  The PyTorch data type tensor is also capable of storing what actions were performed on the data (which is information we will need to calculate gradients later on!)

In [ ]:
import torch 

x_train,y_train,x_valid,y_valid = map(torch.tensor, (x_train,y_train,x_valid,y_valid))

In [ ]:
n,c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()

## Basic model and training loop

Now that our data is loaded as a tensor, we are ready to create a model!  We randomly initialize our weights and normalize by dividing by the sqrt of their size.  We set `weights` to require a gradient, which will record the operations done on the tensor, so that we can calculate the gradient during back-propagation. We do this **after** the random initialization/normalization, since we don't want that step included in the gradient.

We initialize the bias to be all zeros, also setting `requires_grad` to be true.

In [ ]:
import math

weights = torch.randn(784,10)/math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

A model is just a function.  In this case, we will multiply our input by the weights (remember that @ can be used for matrix multiplication in Python 3) and add a set of constants (called the bias).  We then take the log of the softmax.

We will use `torch.nn.functional`. `torch.nn.Module` and `torch.nn.functional` are two key modules in the PyTorch library. The contents of torch.nn.Module are classes, and thus can hold state, whereas the methods in torch.nn.functional do not hold state.

In [ ]:
import torch.nn.functional as F

def model(xb):
    xb = (xb @ weights) + bias
    return F.log_softmax(xb, dim=-1)

bs stands for batch size. [Here is a reference](https://github.com/fastai/fastai_v1/blob/master/docs/abbr.md) for this and other abbreviations used as variable names. (And [here is an explanation](https://github.com/fastai/fastai/blob/master/docs/style.md) of the fast.ai style guide which favors more concise programming than PEP 8)

In [ ]:
bs=64

We will call the function `model` on one batch of data (in this case, 64 images).  This is one *forward pass*.  Note that our predictions won't be very good, since we are just using random weights at this point.

In [ ]:
preds = model(x_train[0:bs])
preds[0], preds.shape

Note that `preds` is a tensor, which contains the tensor values as well as the gradient function.

We chose to use the negative log-likelihood as the loss function for this task:

In [ ]:
loss_fn = F.nll_loss

We now calculate the loss by comparing the predictions to the labels.

In [ ]:
loss_fn(preds, y_train[0:bs])

We set the learning rate and the number of epochs:

In [ ]:
lr = 0.5
epochs = 2

We can now run a training loop.  For each iteration, we will:

- select a mini-batch of data (of size `bs`)
- use the model to make predictions
- calculate the loss
- `loss.backward()` updates the gradients of the model, in this case, `weights` and `bias`  This step is called back-propagation.
- We now use these gradients to update the weights and bias.  We do this within the `torch.no_grad()` context manager, because we do not want these actions to be recorded for our next calculation of the gradient.  You can read more about how PyTorch's Autograd records operations [here](https://pytorch.org/docs/stable/notes/autograd.html).
- We then set the gradients to zero, so that we are ready for the next loop.  Otherwise, our gradients would record a running tally of all the operations that had happened.

Using the IPython debugger gives you a way to step through the loop, allowing  you to check the various variable values at each step. You can uncomment `set_trace()` to try it out.

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

Now we can check the loss again. We expect that the loss will have decreased from before we started the training loop, and it has!  Our predictions are getting better as we update the weights and bias.

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

## Refactor using nn.Module

We will now refactor our code, so that it does the same thing as before, only we'll start taking advantage of PyTorch's `nn` classes. Specifically, we'll use `nn.Module` and `nn.Parameter`, for a clearer and more concise training loop. We subclass `nn.Module` (which itself is a class and able to keep track of state).  In this case, we want to create a class that holds our weights, bias, and method for the forward step.  `nn.Module` has a number of attributes and methods (such as `.parameters()` and `.zero_grad()`) which we will be using.

**NB**: `torch.nn.Module` (uppercase M) is a PyTorch specific concept, and is a class we'll be using a lot. torch.nn.Module is not to be confused with the Python concept of a (lowercase m) [module](https://docs.python.org/3/tutorial/modules.html), which is a file of Python code that can be imported.

(The weights in the class below are initialized with [Xavier initialisation](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) by dividing with 1/sqrt(n).)

In [ ]:
from torch import nn

class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784,10)/math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        xb = (xb @ self.weights) + self.bias
        return F.log_softmax(xb, dim=-1)

We instantiate our model:

In [ ]:
model = Mnist_Logistic()

We calculate the loss in the same way as before. Note that `nn.Module` objects are used as if they are functions (i.e they are *callable*), but behind the scenes Pytorch will call our `forward` method automatically.

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

Now for our training loop.  Before we had to update the values for each parameter by name, and manually zero out the grads for each parameter separately, like this:

```
with torch.no_grad():
    weights -= weights.grad * lr
    bias -= bias.grad * lr
    weights.grad.zero_()
    bias.grad.zero_()
```

Now we can take advantage of model.parameters() and model.zero_grad() (which are both defined by PyTorch for `nn.Module`) to make those steps more concise and less prone to the error of forgetting some of our parameters, particularly if we had a more complicated model:

```
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```

In [ ]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_fn(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
                model.zero_grad()

In [ ]:
fit()

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

## Refactor using nn.Linear

We continue to refactor our code.  Now, instead of manually defining `self.weights` and `self.bias`, and calculating `(xb @ self.weights) + self.bias`, we will instead use the Pytorch class [nn.Linear](https://pytorch.org/docs/stable/nn.html#linear-layers) for a linear layer, which does all that for us. Pytorch has many types of predefined layers that can greatly simplify our code, and often makes it faster too.

In [ ]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784,10)

    def forward(self, xb): return F.log_softmax(self.lin(xb), dim=-1)

We instantiate our model and calculate the loss in the same way as before:

In [ ]:
model = Mnist_Logistic()
loss_fn(model(x_train[0:bs]), y_train[0:bs])

In [ ]:
fit()

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

## Refactor using optim

Pytorch also has a package with various optimization algorithms, `torch.optim`. We can use the `step` method from our optimizer to take a forward step, instead of manually updating each parameter.

This will let us replace our previous:

```
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```

and instead use:

```
opt.step()
opt.zero_grad()
```

`optim.zero_grad()` resets the gradient to 0 and we need to call it before computing the gradient for the next minibatch.

In [ ]:
from torch import optim

In [ ]:
def get_model():
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(), lr=lr)

In [ ]:
model,opt = get_model()
loss_fn(model(x_train[0:bs]), y_train[0:bs])

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

## Refactor using Dataset

Now we use Pytorch's TensorDataset function to build a training dataset.

TensorDataset is a Pytorch utility class that implements the Pytorch Dataset API for a given list of tensors. The Dataset API allows iteration, indexing and slicing along the first dimension of each tensor passed in. This makes it easy to access both the independent and dependent variables in the same line as we train.

[This tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) talks about the Dataset abstract class, how to use it, and how to implement custom datasets for new kinds of data.

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
model = Mnist_Logistic()
opt = optim.SGD(model.parameters(), lr=1.)

In [ ]:
model,opt = get_model()

In [ ]:
train_ds = TensorDataset(x_train, y_train)

In [ ]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        xb,yb = train_ds[i*bs : i*bs+bs]
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

## Refactor using DataLoader

Finally, we use Pytorch's DataLoader to easily iterate over our dataset's minibatches as we train. Rather than having to use `train_ds[i*bs : i*bs+bs]`, the DataLoader gives us each minibatch automatically.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
model,opt = get_model()

In [ ]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

In [ ]:
for epoch in range(epochs):
    for xb,yb in train_dl:
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
loss_fn(model(x_train[0:bs]), y_train[0:bs])

# Add validation

## First try

Next, we will add a validation set for our training.

In this scenario we shuffle the training data but not the validation data. Shuffling the training data is [very important](https://www.quora.com/Does-the-order-of-training-data-matter-when-training-neural-networks) to prevent correlation between batches and overfitting. On the other hand, the validation set is not used for training, only for calculating the loss. The validation loss will be identical whether we shuffle or not and thus it makes no sense to shuffle the validation data.

The batch size for the validation set is twice as large as that for the training set. This is [because](http://forums.fast.ai/t/why-is-batch-size-doubled-for-val-batches/720) the validation set does not need backpropagation and thus takes less memory (it doesn't need to store the gradients). We take advantage of this to use a larger batch size and compute the loss more quickly.

In [ ]:
model,opt = get_model()

In [ ]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs*2)

In [ ]:
loss_fn(model(x_valid[0:bs]), y_valid[0:bs])

In [ ]:
for epoch in range(epochs):
    model.train()
    for xb,yb in train_dl:
        pred = model(xb)
        loss = loss_fn(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
        
    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_fn(model(xb), yb)
                         for xb,yb in valid_dl)

    print(epoch, valid_loss/len(valid_dl))

## Create fit() and get_data()

Now we will create some functions that we will use over and over again when we train models.

`loss_batch` computes the loss for one batch, and optionally also the gradients.

In [ ]:
def loss_batch(model, loss_fn, xb, yb, opt=None):
    loss = loss_fn(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

`fit` runs the necessary operations to train our model and compute the training and validation losses for each epoch.

In [ ]:
import numpy as np

def fit(epochs, model, loss_fn, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl: loss_batch(model, loss_fn, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, loss_fn, xb, yb)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

`get_data` returns dataloaders for the training and validation sets.

In [ ]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True),
            DataLoader(valid_ds, batch_size=bs*2))

train_dl,valid_dl = get_data(train_ds, valid_ds, bs)

In [ ]:
model,opt = get_model()

In [ ]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

# Switch to CNN

## First try

We are now going to build our neural network with three convolutional layers. Since our previous attempt at a neural network was just a simple one linear layer network, we expect a better result (lower loss) in this attempt. 

Because none of the functions in the previous section assume anything about the model form, we'll be able to use them to train a CNN without any modification.

We will use Pytorch's predefined [Conv2d](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) function as our convolutional layer. 

In [ ]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1,1,28,28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        xb = xb.view(-1,xb.size(1))
        return F.log_softmax(xb, dim=-1)

In [ ]:
lr=0.1

We use momentum to help train more quickly.

In [ ]:
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
xb, yb = next(iter(valid_dl))
loss_fn(model(xb), yb)

In [ ]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

## nn.Sequential

Finally, we will integrate all the neural network's operations into a [Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) object. A Sequential object runs each of the functions contained within it, in a sequential manner. This is a simpler way of writing our neural network. It's helpful to define a `Lambda` class to add custom functionality that's not provided by PyTorch, whilst still benefiting from `nn.Sequential`.

In [ ]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func=func
        
    def forward(self, x): return self.func(x)

We can simplify our model a little by replacing the `nll_loss` loss function with `cross_entropy`. `cross_entropy` combines the softmax activation and negative log likelihood loss together into a single function, which results in less code, and is more numerically stable.

In [ ]:
loss_fn = F.cross_entropy

In [ ]:
model = nn.Sequential(
    Lambda(lambda x: x.view(-1,1,28,28)),
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0),-1))
)

In [ ]:
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
xb, yb = next(iter(valid_dl))
loss_fn(model(xb), yb)

In [ ]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)